More complex format
===================

* xml
* html

XML
===
    * data transfer
    * nested elements
    * document validation
    * parsers in most languages
    * focus on bussines
    * 

```xml
<article>
    <info>
        <title>Some title</title>
        <authors>
            <author>
                <firstname>Jakub</firstname>
                <lastname>Kanclerz</lastname>
            </author>
        </authors>
    </info>
    <content>
        <section type="header">
            <paragraph>
                some title
            </paragraph>
        </section>
        <section type="main">
            <paragraph>
                some content
            </paragraph>
            <image src="image/src.jpeg"/>
        </section>
    </content>
</article>
```

Html
====

```html
<li class="link">
    <div class="article" data-id="4789115">
        <a href="#4789115">
            <span>85</span>
            <span class="button button submit">wykop</span>
        </a>
    </div>

    <div class="media-content m-reset-float ">
        <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/">
            <img src="https://www.wykop.pl/cdn/c3397993/link_MwUxfsmJdpfvuiy0ICl1l0Yw35VeWgBc,w207h139.jpg" alt="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">
        </a>
    </div>
    <div class="lcontrast m-reset-margin">
        <h2>
            <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                rel="nofollow noopener noreferrer" title="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">W
                jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.</a>
        </h2>


        <div class="description">
            <p class="text">
                <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                    title="">
                    Tragiczna relacja pracownika, który przez lata obserwował jak serwis promujący wolność słowa
                    zamienił się w ideologiczny rynsztok prześladujący pracowników o zróżnicowanym światopoglądzie.
                </a>
            </p>
        </div>
    </div>
</li>
```

In [23]:
!pip install bs4

You should consider upgrading via the '/Users/jkanclerz/myplace/dydaktyka/przetwarzanie-dokumnetow/data-science-workshops-2021/.venv/bin/python3 -m pip install --upgrade pip' command.


In [24]:
sample_xml = '''
<article>
    <info>
        <title>Some title</title>
        <authors>
            <author>
                <firstname>Jakub</firstname>
                <lastname>Kanclerz</lastname>
            </author>
        </authors>
    </info>
    <content>
        <section type="header">
            <paragraph>
                some title
            </paragraph>
        </section>
        <section type="main">
            <paragraph>
                some content
            </paragraph>
            <image src="image/src.jpeg"/>
        </section>
    </content>
</article>
'''

In [25]:
import requests
from bs4 import BeautifulSoup as bs
from itertools import chain

In [26]:
!pip install lxml

You should consider upgrading via the '/Users/jkanclerz/myplace/dydaktyka/przetwarzanie-dokumnetow/data-science-workshops-2021/.venv/bin/python3 -m pip install --upgrade pip' command.


In [27]:
loaded = bs(sample_xml)

In [28]:
soups = loaded.select('author firstname')

In [29]:
print(soups[0].text)

Jakub


In [30]:
urls = [
    "https://www.wykop.pl/strona/{}".format(page)
    for page in range(1, 10, 1)
]

In [31]:
import collections
from pprint import pprint as pp

Wykop = collections.namedtuple('Wykop', ['count', 'title', 'description', 'tags'])

def clear_text(text):
    return text.rstrip().replace("\n", '').replace("\t", '')

def grab_wykop_item_data(element):
    try:
        tags = map(lambda x: x.text, element.select('.article>div a.tag'))
        tags = filter(lambda x: x.startswith('#'), tags)
        
        return Wykop(
            int(element.select('.article>div a>span:first-child')[0].text),
            element.select('.article>div h2 a')[0]['title'],
            clear_text(element.select('.article>div .description')[0].text),
            list(tags))
    except Exception as e:
        return None

In [32]:
pages = (requests.get(url) for url in urls)
contents = (page.content for page in pages)
soups = (bs(content, 'html.parser') for content in contents)
lists_of_items = (soup.select('#itemsStream li.iC') for soup in soups)
all_items_together = chain.from_iterable(lists_of_items)
extracted = (grab_wykop_item_data(item) for item in all_items_together)
valid = (wykop for wykop in extracted if wykop is not None)

In [33]:
import itertools
first5 = [next(valid) for x in range(4)]

In [34]:
list(first5)

[Wykop(count=263, title='Polityk PiS lata jeździł bez prawka. Prokuratura nie widziała problemu', description='I po to potrzebne są sądy niezależne od rządu. Prokuratura broniła polityka, gdzie powinna oskarżać.', tags=['#polska', '#sady', '#pis', '#prokuratura', '#polityka']),
 Wykop(count=179, title='Adam Małysz ma pomysł na sceptyków. "Powinni dostać przepustkę do szpitala"', description='- Ja się zetknąłem z opinią, że rząd mi zapłacił za ujawnienie swojego zakażenia. Absurd. Zrobiłem to sam z własnej woli. Przeszedłem tę chorobę. Oczywiście nie tak tragicznie, jak niektórzy, ale ja do dziś tego nie lekceważę - mówi Adam Małysz.', tags=['#polska', '#koronawirus', '#covid19', '#zdrowie', '#sport', '#skoki']),
 Wykop(count=375, title='Janusze z Allegro nacinają ludzi na chińskie koncentratory tlenu w cenie 2.5x', description='Janusze biznesu żerują na Allegro na zdesperowanych ludziach sprzedając po kilkukrotnie zawyżonej cenie domowe koncentratory tlenu z chin przydatne w przechodze

In [35]:
tags = (wykop.tags for wykop in first5)

In [36]:
all_tags = list(chain.from_iterable(tags))

In [37]:
counter = {}

for tag in all_tags:
    if tag not in counter.keys():
        counter[tag] = 1
    else:
        counter[tag] = counter[tag] + 1

In [38]:
counter

{'#polska': 4,
 '#sady': 1,
 '#pis': 1,
 '#prokuratura': 1,
 '#polityka': 1,
 '#koronawirus': 2,
 '#covid19': 1,
 '#zdrowie': 1,
 '#sport': 1,
 '#skoki': 1,
 '#januszebiznesu': 1,
 '#prawo': 1,
 '#praworzadnosc': 1,
 '#bekazpisu': 1,
 '#neuropa': 1,
 '#bekazprawakow': 1}

In [39]:
import operator
list(reversed(sorted(counter.items(), key=lambda item: item[1])))

[('#polska', 4),
 ('#koronawirus', 2),
 ('#bekazprawakow', 1),
 ('#neuropa', 1),
 ('#bekazpisu', 1),
 ('#praworzadnosc', 1),
 ('#prawo', 1),
 ('#januszebiznesu', 1),
 ('#skoki', 1),
 ('#sport', 1),
 ('#zdrowie', 1),
 ('#covid19', 1),
 ('#polityka', 1),
 ('#prokuratura', 1),
 ('#pis', 1),
 ('#sady', 1)]

In [40]:
from collections import Counter
c = Counter(all_tags)
c.most_common()

[('#polska', 4),
 ('#koronawirus', 2),
 ('#sady', 1),
 ('#pis', 1),
 ('#prokuratura', 1),
 ('#polityka', 1),
 ('#covid19', 1),
 ('#zdrowie', 1),
 ('#sport', 1),
 ('#skoki', 1),
 ('#januszebiznesu', 1),
 ('#prawo', 1),
 ('#praworzadnosc', 1),
 ('#bekazpisu', 1),
 ('#neuropa', 1),
 ('#bekazprawakow', 1)]

persistance
-------------

In [41]:
import csv
with open('wykop.csv', 'w') as destination_file:
    writer = csv.writer(destination_file, delimiter=';', quotechar='"')
    for row in first5:
        writer.writerow(row)

In [42]:
import json
with open('wykop.data', 'w') as destination_file:
    for row in first5:
        destination_file.write(json.dumps(row._asdict()) + "\n")